# Working with Sparse Layouts in the GraphBLAS Dialect

This example will go over how to use the `--graphblas-lower` pass from `graphblas-opt` to lower the GraphBLAS dialect ops that directly manipulate the layouts of sparse tensors. In particular, we'll focus on the `graphblas.convert_layout` and `graphblas.transpose` ops.

Since the [ops reference](../../ops_reference.rst) already documents these ops with examples, we'll only briefly describe them here. 

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [2]:
import mlir_graphblas
from mlir_graphblas.tools.utils import sparsify_array, densify_csr, densify_csc
import numpy as np

engine = mlir_graphblas.MlirJitEngine()

## Overview of graphblas.convert_layout

Here, we'll show how to use the `graphblas.convert_layout` op. 

This op takes 1 sparse matrix in CSR or CSC format and creates a new sparse matrix of the desired format.

We'll give several examples below of how this will work.

First, we'll define an example input CSR matrix.

In [2]:
dense_matrix = np.array(
    [
        [1.1, 0. , 0. , 0. ],
        [0. , 0. , 2.2, 0. ],
        [0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. ]
    ],
    dtype=np.float64,
)

csr_matrix = sparsify_array(dense_matrix, [False, True])

## graphblas.convert_layout (CSR->CSC)

Let's convert this matrix to CSC format.

In [3]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csr_to_csc(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}
"""

Here are the passes we'll use.

In [4]:
passes = [
    "--graphblas-lower",
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-bufferize",
    "--tensor-constant-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-memref-to-llvm",
    "--convert-std-to-llvm",
]

In [5]:
engine.add(mlir_text, passes)

['csr_to_csc']

In [6]:
csc_matrix = engine.csr_to_csc(csr_matrix)

In [7]:
densify_csc(csc_matrix)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

In [8]:
np.all(dense_matrix == densify_csc(csc_matrix))

True

## graphblas.convert_layout (CSC->CSR)

Let's convert the CSC matrix back to CSR format.

Let's first get rid of our original `csr_matrix` so we don't get correct results purely by accident.

In [9]:
del csr_matrix

Here's the MLIR code to convert from CSC to CSR. 

In [10]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csc_to_csr(%sparse_tensor: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSR64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSC64> to tensor<?x?xf64, #CSR64>
    return %answer : tensor<?x?xf64, #CSR64>
}
"""

In [11]:
engine.add(mlir_text, passes)

['csc_to_csr']

In [12]:
csr_matrix = engine.csc_to_csr(csc_matrix)

In [13]:
densify_csr(csr_matrix)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

In [14]:
np.all(dense_matrix == densify_csr(csr_matrix))

True

## graphblas.convert_layout (CSC->CSC, CSR->CSR)

For completeness, we'll show how to convert to and from the same exact layouts.

The MLIR code to do so is shown below.

In [15]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @csc_to_csc(%sparse_tensor: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSC64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}

func @csr_to_csr(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSR64> {
    %answer = graphblas.convert_layout %sparse_tensor : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSR64>
    return %answer : tensor<?x?xf64, #CSR64>
}
"""

In [16]:
engine.add(mlir_text, passes)

['csc_to_csc', 'csr_to_csr']

Let's verify that converting to and from the same layout give correct results.

In [17]:
csc_result = engine.csc_to_csc(csc_matrix)

In [18]:
densify_csc(csc_result)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

In [19]:
np.all(dense_matrix == densify_csc(csc_result))

True

In [20]:
csr_result = engine.csr_to_csr(csr_matrix)

In [21]:
densify_csr(csr_result)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

In [22]:
np.all(dense_matrix == densify_csr(csr_result))

True

## Overview of graphblas.transpose

Here, we'll show how to use the `graphblas.transpose` op. 

`graphblas.transpose` returns a new sparse matrix that’s the transpose of the input matrix. Note that the behavior of this op differs depending on the sparse encoding of the specified output tensor type.

The input/output behavior of `graphblas.transpose` is fairly simple. Our examples here aren't intended to show anything interesting but to merely act as reproducible references.

The important thing to know about `graphblas.transpose` is how it is implemented.

When transposing a CSC matrix to a CSR matrix, we simply need to swap the dimension sizes and reverse the indexing. Thus, the only "real" work done here is changing metadata. The same goes for transposing a CSC matrix to a CSR matrix.

Here's an example of transposing a CSR matrix to a CSC matrix.

In [23]:
dense_matrix = np.array(
    [
        [1.1, 0. , 0. , 0. ],
        [0. , 0. , 2.2, 0. ],
    ],
    dtype=np.float64,
)

csr_matrix = sparsify_array(dense_matrix, [False, True])

In [24]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @transpose_csr_to_csc(%sparse_tensor: tensor<?x?xf64, #CSR64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.transpose %sparse_tensor : tensor<?x?xf64, #CSR64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}
"""

In [25]:
engine.add(mlir_text, passes)

['transpose_csr_to_csc']

In [26]:
csc_matrix_transpose = engine.transpose_csr_to_csc(csr_matrix)

In [27]:
densify_csc(csc_matrix_transpose)

array([[1.1, 0. ],
       [0. , 0. ],
       [0. , 2.2],
       [0. , 0. ]])

In [28]:
np.all(dense_matrix.T == densify_csc(csc_matrix_transpose))

True

However, when we're transposing a CSR matrix and want to return a CSR matrix as well, there is "real" work that is done. This "real" work involves doing exactlly what `graphblas.convert_layout` does under the covers in addition to changing the metadata. The same goes for transposing a CSC matrix to a CSC matrix. 

The example below shows how to transpose a CSC mmatrix to a CSC matrix. 

In [29]:
mlir_text = """
#CSR64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (i,j)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

#CSC64 = #sparse_tensor.encoding<{
  dimLevelType = [ "dense", "compressed" ],
  dimOrdering = affine_map<(i,j) -> (j,i)>,
  pointerBitWidth = 64,
  indexBitWidth = 64
}>

func @transpose_csc_to_csc(%sparse_tensor: tensor<?x?xf64, #CSC64>) -> tensor<?x?xf64, #CSC64> {
    %answer = graphblas.transpose %sparse_tensor : tensor<?x?xf64, #CSC64> to tensor<?x?xf64, #CSC64>
    return %answer : tensor<?x?xf64, #CSC64>
}
"""

In [30]:
engine.add(mlir_text, passes)

['transpose_csc_to_csc']

In [31]:
csc_matrix = engine.transpose_csc_to_csc(csc_matrix_transpose)

In [32]:
densify_csc(csc_matrix)

array([[1.1, 0. , 0. , 0. ],
       [0. , 0. , 2.2, 0. ]])

In [33]:
np.all(dense_matrix == densify_csc(csc_matrix))

True